In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import models, layers

In [2]:
rooth_path = 'data/'
deng_sj = pd.read_csv(rooth_path + 'deng_sj.csv')
deng_iq = pd.read_csv(rooth_path + 'deng_iq.csv')
test_sj = pd.read_csv(rooth_path + 'test_sj.csv')
test_iq = pd.read_csv(rooth_path + 'test_iq.csv')

In [ ]:
def convert_celsius(df):
    
    k = 273.15
    
    # convert to celsius
    df['reanalysis_air_temp_k'] -= k
    df['reanalysis_avg_temp_k'] -= k
    df['reanalysis_dew_point_temp_k'] -= k
    df['reanalysis_max_air_temp_k'] -= k
    df['reanalysis_min_air_temp_k'] -= k
    
    # change features names
    col = df.columns
    col = col.str.replace('temp_k', 'temp_c')
    df.columns = col.str.replace('tdtr_k', 'tdtr_c')
    
    return df

In [ ]:
deng_sj = convert_celsius(deng_sj)
deng_iq = convert_celsius(deng_iq)
test_sj = convert_celsius(test_sj)
test_iq = convert_celsius(test_iq)

In [ ]:
def mean_ndvi(df):
    
    df['ndvi_mean'] = df[df.columns[3:7]].mean(axis = 1)
    
    return df

In [ ]:
deng_sj = mean_ndvi(deng_sj)
deng_iq = mean_ndvi(deng_iq)
test_sj = mean_ndvi(test_sj)
test_iq = mean_ndvi(test_iq)

In [ ]:
def feature_selection(df, columns):
    
    df.drop(columns = columns, inplace=True)
    
    return df

In [ ]:
deng_sj = feature_selection(deng_sj, ['reanalysis_air_temp_c', 'reanalysis_sat_precip_amt_mm', 'reanalysis_dew_point_temp_c'])
deng_iq = feature_selection(deng_iq, ['reanalysis_avg_temp_c', 'reanalysis_sat_precip_amt_mm', 'reanalysis_dew_point_temp_c'])
test_sj = feature_selection(test_sj, ['reanalysis_air_temp_c', 'reanalysis_sat_precip_amt_mm', 'reanalysis_dew_point_temp_c'])
test_iq = feature_selection(test_iq, ['reanalysis_avg_temp_c', 'reanalysis_sat_precip_amt_mm', 'reanalysis_dew_point_temp_c'])

In [ ]:
def rolling_feat(df, df_test, roll_win_size, city):
    
    # save total cases in a dataframe
    total_cases = df[['city', 'year', 'weekofyear', 'total_cases']]
    
    df.drop(columns=['total_cases'], inplace=True)
    
    total_df = pd.concat([df, df_test], axis=0)
    
    cols = total_df.columns[4:] #Features columns excluding week_start_date, city and weekofyear
    
    for col in cols:
        total_df[col + '_sum']  = total_df[col].rolling(roll_win_size).sum()
        total_df[col + '_av'] = total_df[col].rolling(roll_win_size).mean()
        total_df[col + '_var']  = total_df[col].rolling(roll_win_size).var()
        
    # Split train-test again
    
    if city == 'San Juan':
        df_ = total_df[total_df.week_start_date < '2008-04-29']
        df_test = total_df[total_df.week_start_date >= '2008-04-29']
    
    if city == 'Iquitos':
        df_ = total_df[total_df.week_start_date < '2010-07-02']
        df_test = total_df[total_df.week_start_date >= '2010-07-02']
    
    df_ = df_.merge(total_cases, on=['city', 'year', 'weekofyear'])
      
    df_.dropna(axis=0, inplace=True) #trim rows with null values of rolling features
    
    return df_, df_test

In [ ]:
deng_sj_train, test_sj = rolling_feat(deng_sj, test_sj, 4, 'San Juan')
deng_iq_train, test_iq = rolling_feat(deng_iq, test_iq, 4, 'Iquitos')

In [ ]:
def enc_quarters(df):
    
    df['week_start_date'] = pd.to_datetime(df.week_start_date)
    df['quarter'] = df['week_start_date'].dt.quarter
    
    df = pd.get_dummies(df, columns=['quarter'])
    
    return df

In [ ]:
deng_sj_train = enc_quarters(deng_sj_train)
deng_iq_train = enc_quarters(deng_iq_train)
test_sj = enc_quarters(test_sj)
test_iq = enc_quarters(test_iq)

In [ ]:
def split_df(df):
    
    split = df.shape[0] - df.shape[0]*0.1 # Split with train 90% and validation 10%
    
    df_train = df[df.index < split]
    df_val = df[df.index >= split]
    
    
    return df_train, df_val

In [ ]:
deng_sj_train, deng_sj_val = split_df(deng_sj_train) #SJ split data in train-validation
deng_iq_train, deng_iq_val = split_df(deng_iq_train) #IQ split data in train-validation

In [ ]:
# LAYERS
def init_model():

    model = models.Sequential()
    model.add(layers.LSTM(20, return_sequences=True, activation='tanh'))
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.LSTM(10, return_sequences=True, activation='tanh'))
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.LSTM(10, activation='tanh'))
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(1, activation='linear'))
    
    model.compile(loss='mse', 
                  optimizer='rmsprop', 
                  metrics=['mae'])
    return model

model = init_model()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train,
            validation_split=0.15,
            epochs=1000, 
            batch_size=16,
            callbacks=[es])
model.evaluate(X_test, y_test)

In [ ]:
def plot_loss_mae(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='best')
    plt.show()
    
    plt.plot(history.history['mae'])
    plt.plot(history.history['val_mae'])
    plt.title('Model Mean Absolute Error')
    plt.ylabel('Mean Absolute Error')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='best')
    plt.show()